# **Maestría en Inteligencia Artificial Aplicada**

## Curso: **Análisis de grandes volúmenes de datos (Gpo 10)**

### Tecnológico de Monterrey

## Actividad 3

###  Aprendizaje supervisado y no supervisado


#### **Nombrey matrícula**

*   **A01746998** - Alexys Martín Coate Reyes


# **1. Introducción teórica**



## **Aprendizaje Supervisado**

El aprendizaje supervisado intenta predecir un output con base en una serie de datos de entrenamiento que vienen previamente clasificados de manera correcta. Este tipo de aprendizaje se asemeja al que tiene un niño pequeño que está aprendiendo a abstraer las características por medio de una guía, padre o profesor.

Entre los algoritmos supervisados más populares están los siguientes:

* Regresión Lineal
  * ***pyspark.ml.regression.LinearRegression***
* Regresión Logística
  * ***pyspark.ml.classification.LogisticRegression***
* Árboles de Decisión
  * ***pyspark.ml.classification.DecisionTreeClassifier***
  * ***pyspark.ml.regression.DecisionTreeRegressor***
* Bosques Aleatorios (Random Forests)
  * ***pyspark.ml.classification.RandomForestClassifier***
  * ***pyspark.ml.regression.RandomForestRegressor***
* Máquinas de Soporte Vectorial (SVM)
  * ***pyspark.ml.classification.LinearSVC***
* Gradient-Boosted Trees (GBTs)
  * ***pyspark.ml.classification.GBTClassifier***
  * ***pyspark.ml.regression.GBTRegressor***
* Bayes Ingenuo (Naive Bayes)
  * ***pyspark.ml.classification.NaiveBayes***


## **Aprendizaje No Supervisado**

Este tipo de modelos trabajan con un conjunto de datos no etiquetado, por lo que el mismo modelo aprende de manera automática los patrones y relaciones ocultas por si mismo.

Comunmente se utilizan en problemas de "clustering", "Reducción de dimensionalidad" o "Reglas de asociación".

Ejemplos de estos modelos son:

* K-Means
  * ***pyspark.ml.clustering.KMeans***
* Análisis de Componentes Principales (PCA)
  * ***pyspark.ml.feature.PCA***
* Factorización de Matrices No Negativas (NMF)
  * ***pyspark.ml.clustering.NNMF***
* Gaussian Mixture Models (GMM)
  * ***pyspark.ml.clustering.GaussianMixture***

# **2. Selección de los datos**

### Cargando los datos

In [ ]:
# Librerias
from pyspark.sql import SparkSession

In [114]:
# Crear sesión Spark
spark = SparkSession.builder \
    .appName("EDA_Vuelos") \
    .getOrCreate()

# Leer el CSV
df = spark.read.csv("./Airline_Delay_2016-2018.csv", header=True, inferSchema=True)

# Mostrar esquema de datos
df.printSchema()

# Número total de registros
total_registros = df.count()
print(f"Número total de registros: {total_registros}")

root
 |-- FL_DATE: date (nullable = true)
 |-- OP_CARRIER: string (nullable = true)
 |-- OP_CARRIER_FL_NUM: integer (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- CRS_DEP_TIME: integer (nullable = true)
 |-- DEP_TIME: double (nullable = true)
 |-- DEP_DELAY: double (nullable = true)
 |-- TAXI_OUT: double (nullable = true)
 |-- WHEELS_OFF: double (nullable = true)
 |-- WHEELS_ON: double (nullable = true)
 |-- TAXI_IN: double (nullable = true)
 |-- CRS_ARR_TIME: integer (nullable = true)
 |-- ARR_TIME: double (nullable = true)
 |-- ARR_DELAY: double (nullable = true)
 |-- CANCELLED: double (nullable = true)
 |-- DIVERTED: double (nullable = true)
 |-- CRS_ELAPSED_TIME: double (nullable = true)
 |-- ACTUAL_ELAPSED_TIME: double (nullable = true)
 |-- AIR_TIME: double (nullable = true)
 |-- DISTANCE: double (nullable = true)

Número total de registros: 18505725


In [115]:
# Imprimiendo los 3 primeros rengloes del dataframe dataframe
df.show(3)

+----------+----------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+--------+----------------+-------------------+--------+--------+
|   FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|DEST|CRS_DEP_TIME|DEP_TIME|DEP_DELAY|TAXI_OUT|WHEELS_OFF|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|ARR_DELAY|CANCELLED|DIVERTED|CRS_ELAPSED_TIME|ACTUAL_ELAPSED_TIME|AIR_TIME|DISTANCE|
+----------+----------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+--------+----------------+-------------------+--------+--------+
|2016-01-01|        DL|             1248|   DTW| LAX|        1935|  1935.0|      0.0|    23.0|    1958.0|   2107.0|   13.0|        2144|  2120.0|    -24.0|      0.0|     0.0|           309.0|              285.0|   249.0|  1979.0|
|2016-01-01|        DL|             1251|   ATL| GRR|        2125|  2130.0|     

### Seleccionando variables de caracterización

In [116]:
# Variables seleccionadas
vars_particion = ["OP_CARRIER", "ORIGIN", "DEST", "CANCELLED", "DIVERTED"]

# Imrpimiendo la cantidad de valores únicos que se tiene por las columnas seleccionadas
for col in vars_particion:
    print(col, df.select(col).distinct().count())

OP_CARRIER 18
ORIGIN 362
DEST 360
CANCELLED 2
DIVERTED 2


In [117]:
from pyspark.sql.functions import col, count, round

# Calcular frecuencia de combinaciones
combinaciones = df.groupBy(vars_particion).count()

# Total de registros
total = df.count()

# Agregar probabilidad
combinaciones = combinaciones.withColumn("probabilidad", col("count") / total)

combinaciones.show(truncate=False)

+----------+------+----+---------+--------+-----+---------------------+
|OP_CARRIER|ORIGIN|DEST|CANCELLED|DIVERTED|count|probabilidad         |
+----------+------+----+---------+--------+-----+---------------------+
|EV        |ATL   |AGS |0.0      |0.0     |2912 |1.573567098830227E-4 |
|EV        |DTW   |BWI |0.0      |0.0     |23   |1.2428586288837644E-6|
|EV        |BRO   |IAH |0.0      |0.0     |3054 |1.6503001098308766E-4|
|F9        |DEN   |MIA |0.0      |0.0     |1089 |5.884665421106171E-5 |
|WN        |BDL   |RSW |0.0      |0.0     |735  |3.971743879258986E-5 |
|WN        |BWI   |MEM |0.0      |0.0     |1348 |7.284232311892671E-5 |
|WN        |DEN   |BOS |0.0      |0.0     |2467 |1.3331009728070637E-4|
|WN        |LAX   |LAS |0.0      |0.0     |10922|5.901957367247162E-4 |
|WN        |LAX   |SAT |0.0      |0.0     |2056 |1.1110075395587041E-4|
|WN        |MCO   |CMH |0.0      |0.0     |3558 |1.922648261551493E-4 |
|WN        |PIT   |DEN |0.0      |0.0     |1307 |7.0626792519612

In [118]:
from pyspark.sql.functions import when, lit

# Definir tamaño total de la muestra (por ejemplo, 1% del total)
tamaño_muestra_total = int(total * 0.01)

# Establecer un mínimo de registros por partición
minimo_por_particion = 0

# Calcular el tamaño de muestra por partición según su probabilidad
combinaciones_con_tamaño = combinaciones.withColumn(
    "tamaño_muestra",
    round(
        when(
            (col("probabilidad") * tamaño_muestra_total) < minimo_por_particion,
            lit(minimo_por_particion)
        ).otherwise(col("probabilidad") * tamaño_muestra_total)
    ).cast("int")
)

# Ordenar para visualizar mejor
combinaciones_con_tamaño = combinaciones_con_tamaño.orderBy(col("tamaño_muestra").desc())

combinaciones_con_tamaño.show(10, truncate=False)

# Calcular el tamaño final total de la muestra
tamaño_muestra_final = combinaciones_con_tamaño.agg({"tamaño_muestra": "sum"}).collect()[0][0]

print(f"Tamaño total esperado de la muestra final: {tamaño_muestra_final}")

+----------+------+----+---------+--------+-----+---------------------+--------------+
|OP_CARRIER|ORIGIN|DEST|CANCELLED|DIVERTED|count|probabilidad         |tamaño_muestra|
+----------+------+----+---------+--------+-----+---------------------+--------------+
|HA        |OGG   |HNL |0.0      |0.0     |28891|0.0015611925498730799|289           |
|HA        |HNL   |OGG |0.0      |0.0     |28830|0.0015578962726399534|288           |
|HA        |KOA   |HNL |0.0      |0.0     |21350|0.0011536970315942769|213           |
|HA        |HNL   |KOA |0.0      |0.0     |20673|0.0011171137580397417|207           |
|HA        |LIH   |HNL |0.0      |0.0     |19901|0.0010753969379745998|199           |
|HA        |HNL   |LIH |0.0      |0.0     |19946|0.0010778286179006767|199           |
|WN        |DAL   |HOU |0.0      |0.0     |19109|0.0010325993712756458|191           |
|WN        |HOU   |DAL |0.0      |0.0     |19131|0.0010337881925728389|191           |
|DL        |MCO   |ATL |0.0      |0.0     |

In [119]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

# Unir el tamaño de muestra a cada combinación en el dataset original
df_con_muestra = df.join(
    combinaciones_con_tamaño.select(*vars_particion, "tamaño_muestra"),
    on=vars_particion,
    how="inner"
)

# Crear ventana ordenada por fecha y hora dentro de cada combinación
ventana = Window.partitionBy(*vars_particion).orderBy("FL_DATE", "CRS_DEP_TIME")

# Enumerar vuelos por combinación (ordenados por tiempo)
df_con_muestra = df_con_muestra.withColumn("row_num", row_number().over(ventana))

# Filtrar solo los primeros N registros por combinación
muestra_final = df_con_muestra.filter(col("row_num") <= col("tamaño_muestra"))

# Mostrar la muestra final
muestra_final.show(10, truncate=False)

print(f"Tamaño total de la muestra: {muestra_final.count()}")

+----------+------+----+---------+--------+----------+-----------------+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+----------------+-------------------+--------+--------+--------------+-------+
|OP_CARRIER|ORIGIN|DEST|CANCELLED|DIVERTED|FL_DATE   |OP_CARRIER_FL_NUM|CRS_DEP_TIME|DEP_TIME|DEP_DELAY|TAXI_OUT|WHEELS_OFF|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|ARR_DELAY|CRS_ELAPSED_TIME|ACTUAL_ELAPSED_TIME|AIR_TIME|DISTANCE|tamaño_muestra|row_num|
+----------+------+----+---------+--------+----------+-----------------+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+----------------+-------------------+--------+--------+--------------+-------+
|9E        |ATL   |AGS |0.0      |0.0     |2018-01-01|3296             |855         |854.0   |-1.0     |12.0    |906.0     |939.0    |3.0    |953         |942.0   |-11.0    |58.0            |48.0               |33.0    |143.0   |10          

# **3. Preparación de los datos**

## **Modelo Supervisado**

### Selección de variables importantes para el modelo

Se realizará un análisis de predicción para saber que tanto un avión se retrasará. Por ello se quitarán las columnas que no aportan mucha información para el modelo.

In [120]:
import pandas as pd

pandas_df = muestra_final.limit(50).toPandas()
display(pandas_df)

,OP_CARRIER,ORIGIN,DEST,CANCELLED,DIVERTED,FL_DATE,OP_CARRIER_FL_NUM,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,...,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,tamaño_muestra,row_num
0,9E,ATL,AGS,0.0,0.0,2018-01-01,3296,855,854.0,-1.0,...,3.0,953,942.0,-11.0,58.0,48.0,33.0,143.0,10,1
1,9E,ATL,AGS,0.0,0.0,2018-03-01,3680,1355,1534.0,99.0,...,3.0,1445,1636.0,111.0,50.0,62.0,45.0,143.0,10,2
2,9E,ATL,AGS,0.0,0.0,2018-03-02,3366,1746,1752.0,6.0,...,3.0,1845,1843.0,-2.0,59.0,51.0,30.0,143.0,10,3
3,9E,ATL,AGS,0.0,0.0,2018-03-02,3809,2134,2132.0,-2.0,...,5.0,2232,2226.0,-6.0,58.0,54.0,30.0,143.0,10,4
4,9E,ATL,AGS,0.0,0.0,2018-03-03,3354,1026,1019.0,-7.0,...,4.0,1122,1115.0,-7.0,56.0,56.0,31.0,143.0,10,5
5,9E,ATL,AGS,0.0,0.0,2018-03-03,3366,1746,1826.0,40.0,...,5.0,1842,1913.0,31.0,56.0,47.0,30.0,143.0,10,6
6,9E,ATL,AGS,0.0,0.0,2018-03-03,3904,2234,2228.0,-6.0,...,4.0,2327,2323.0,-4.0,53.0,55.0,35.0,143.0,10,7
7,9E,ATL,AGS,0.0,0.0,2018-03-04,3366,1746,1745.0,-1.0,...,4.0,1845,1837.0,-8.0,59.0,52.0,27.0,143.0,10,8
8,9E,ATL,AGS,0.0,0.0,2018-03-04,3809,2134,2130.0,-4.0,...,4.0,2232,2222.0,-10.0,58.0,52.0,29.0,143.0,10,9
9,9E,ATL,AGS,0.0,0.0,2018-03-05,3366,1746,1749.0,3.0,...,4.0,1845,1843.0,-2.0,59.0,54.0,29.0,143.0,10,10


In [121]:
# Se crea una nueva columna para verificar si
delayed_time = 10
df_muestra = muestra_final.withColumn("DELAYED", when(df.ARR_DELAY > delayed_time, 1).otherwise(0))

In [122]:
# Variables a descartar
cols_to_drop = ["tamaño_muestra", "row_num", "OP_CARRIER_FL_NUM", "DEP_TIME","DEP_DELAY", "TAXI_OUT",
                "WHEELS_OFF", "WHEELS_ON", "TAXI_IN" , "ARR_TIME", "ARR_DELAY", "DIVERTED", "ACTUAL_ELAPSED_TIME", "AIR_TIME"]

# Variables importantes para el modelo
selected_cols = [
    "FL_DATE", "OP_CARRIER", "ORIGIN", "DEST",
    "CRS_DEP_TIME", "CRS_ARR_TIME", "CRS_ELAPSED_TIME",
    "DISTANCE", "CANCELLED", "ARR_DELAY", "DELAYED"
]

# Realizando la selección de columnas importantes como datos de entrenamiento
df_raw = df_muestra.select(*selected_cols)
df_raw.show()

+----------+----------+------+----+------------+------------+----------------+--------+---------+---------+-------+
|   FL_DATE|OP_CARRIER|ORIGIN|DEST|CRS_DEP_TIME|CRS_ARR_TIME|CRS_ELAPSED_TIME|DISTANCE|CANCELLED|ARR_DELAY|DELAYED|
+----------+----------+------+----+------------+------------+----------------+--------+---------+---------+-------+
|2018-01-01|        9E|   AGS| ATL|        1018|        1130|            72.0|   143.0|      0.0|     30.0|      1|
|2018-03-01|        9E|   AGS| ATL|        1520|        1621|            61.0|   143.0|      0.0|    126.0|      1|
|2018-03-02|        9E|   AGS| ATL|        1910|        2009|            59.0|   143.0|      0.0|      2.0|      0|
|2018-03-03|        9E|   AGS| ATL|         820|         925|            65.0|   143.0|      0.0|    -14.0|      0|
|2018-03-03|        9E|   AGS| ATL|        1147|        1253|            66.0|   143.0|      0.0|    -16.0|      0|
|2018-03-03|        9E|   AGS| ATL|        1910|        2009|           

### Transformación de variables

In [123]:
from pyspark.sql.functions import dayofweek, month
from pyspark.ml.feature import StringIndexer

# Extrayendo el día, mes y eliminando la columna original de fecha
df_transformed = df_raw.withColumn("FL_DAY_OF_WEEK", dayofweek("FL_DATE")) \
                     .withColumn("FL_MONTH", month("FL_DATE")) \
                     .drop("FL_DATE")

# Codificaando las variables categóricas
indexers = [
    StringIndexer(inputCol="OP_CARRIER", outputCol="OP_CARRIER_T"),
    StringIndexer(inputCol="ORIGIN", outputCol="ORIGIN_T"),
    StringIndexer(inputCol="DEST", outputCol="DEST_T")
]

for indexer in indexers:
    df_transformed = indexer.fit(df_transformed).transform(df_transformed)

# Eliminando las columnas de las variables categóricas originales
cols_to_drop = ["OP_CARRIER", "ORIGIN", "DEST"]
df_transformed = df_transformed.drop(*cols_to_drop) # Using * to unpack the list of column names

df_transformed.show()

+------------+------------+----------------+--------+---------+---------+-------+--------------+--------+------------+--------+------+
|CRS_DEP_TIME|CRS_ARR_TIME|CRS_ELAPSED_TIME|DISTANCE|CANCELLED|ARR_DELAY|DELAYED|FL_DAY_OF_WEEK|FL_MONTH|OP_CARRIER_T|ORIGIN_T|DEST_T|
+------------+------------+----------------+--------+---------+---------+-------+--------------+--------+------------+--------+------+
|        1018|        1130|            72.0|   143.0|      0.0|     30.0|      1|             2|       1|        14.0|   155.0|   0.0|
|        1520|        1621|            61.0|   143.0|      0.0|    126.0|      1|             5|       3|        14.0|   155.0|   0.0|
|        1910|        2009|            59.0|   143.0|      0.0|      2.0|      0|             6|       3|        14.0|   155.0|   0.0|
|         820|         925|            65.0|   143.0|      0.0|    -14.0|      0|             7|       3|        14.0|   155.0|   0.0|
|        1147|        1253|            66.0|   143.0|  

In [124]:
# Guardando dataset en un archivo paquet para su posterior utilización
df_transformed.write.mode("overwrite").parquet("./df_transformed.parquet")

In [125]:
# Lectura del archivo que contiene el dataset de pyspark
df_transformed = spark.read.parquet("./df_transformed.parquet")

In [126]:
# Imprime los valores resultantes de la limpieza y transformación de los datos
df_transformed.printSchema()

root
 |-- CRS_DEP_TIME: integer (nullable = true)
 |-- CRS_ARR_TIME: integer (nullable = true)
 |-- CRS_ELAPSED_TIME: double (nullable = true)
 |-- DISTANCE: double (nullable = true)
 |-- CANCELLED: double (nullable = true)
 |-- ARR_DELAY: double (nullable = true)
 |-- DELAYED: integer (nullable = true)
 |-- FL_DAY_OF_WEEK: integer (nullable = true)
 |-- FL_MONTH: integer (nullable = true)
 |-- OP_CARRIER_T: double (nullable = true)
 |-- ORIGIN_T: double (nullable = true)
 |-- DEST_T: double (nullable = true)



In [127]:
# Imprime un resumen del dataframe con la limpieza de todos los datos
df_transformed.describe().toPandas()

,summary,CRS_DEP_TIME,CRS_ARR_TIME,CRS_ELAPSED_TIME,DISTANCE,CANCELLED,ARR_DELAY,DELAYED,FL_DAY_OF_WEEK,FL_MONTH,OP_CARRIER_T,ORIGIN_T,DEST_T
0,count,183519,183519,183519,183519,183519,181725,183519,183519,183519,183519,183519,183519
1,mean,1326.692860139822,1490.1785046779898,145.20731913316877,832.4616960641678,0.00941046976062424,4.252309808776999,0.22380244007432473,3.939831843024428,1.3503397468382021,3.5820541742271916,32.71207340929277,32.706695219568545
2,stddev,490.4683401486879,513.2910568253734,76.68175372368177,613.7268529053202,0.09655031649425486,43.57478708655211,0.41679233974906305,2.0846753646942195,1.3889923992291628,3.856754148605553,45.59067431687259,45.562444046831985
3,min,1,1,21.0,31.0,0.0,-70.0,0,1,1,0.0,0.0,0.0
4,max,2359,2359,705.0,4983.0,1.0,1568.0,1,7,12,17.0,356.0,356.0


### Limpieza de datos

In [128]:
df_transformed.groupBy("CANCELLED").count().show()  # Cuenta cuantos vuelos cancelados hay
df_transformed.groupBy("FL_MONTH").count().show()   # Cuenta cuantos meses diferentes hay

+---------+------+
|CANCELLED| count|
+---------+------+
|      0.0|181792|
|      1.0|  1727|
+---------+------+

+--------+------+
|FL_MONTH| count|
+--------+------+
|      12|   337|
|       1|167998|
|       6|  2621|
|       3|  2206|
|       5|  1295|
|       9|   424|
|       4|  3729|
|       8|   744|
|       7|   488|
|      10|   517|
|      11|   818|
|       2|  2342|
+--------+------+



In [129]:
# Se elimina la columna de Cancelled y del mes ya que unicamente se cuenta con datos de un solo mes y no existen vuelos cancelasdos
drop_cols = ["CANCELLED", "FL_MONTH"]
df_transformed_2 = df_transformed.drop(*drop_cols)

In [130]:
#Se eliminan registros con valores nulos
df_clean = df_transformed_2.dropna()

#Se eliminan columnas con valores nulos
df_clean = df_clean.na.drop()

#Se eliminan registros duplicados
df_clean = df_clean.dropDuplicates()

In [131]:
df_clean.describe().toPandas()

,summary,CRS_DEP_TIME,CRS_ARR_TIME,CRS_ELAPSED_TIME,DISTANCE,ARR_DELAY,DELAYED,FL_DAY_OF_WEEK,OP_CARRIER_T,ORIGIN_T,DEST_T
0,count,181293,181293,181293,181293,181293,181293,181293,181293,181293,181293
1,mean,1327.197558648155,1490.635722283817,145.47464601501437,834.8054861467349,4.281897260236192,0.22642352435008523,3.9334557870408675,3.573866613713712,32.67865830451259,32.70636483482539
2,stddev,490.43053946338205,513.3674900185148,76.76904671764814,614.4108062889057,43.61791200708378,0.41851747648638965,2.0848308050711406,3.85227977839103,45.5693921884905,45.57088804416217
3,min,1,1,21.0,31.0,-70.0,0,1,0.0,0.0,0.0
4,max,2359,2359,705.0,4983.0,1568.0,1,7,17.0,356.0,356.0


### Balanceo del dataset

In [132]:
df_clean.groupBy("DELAYED").count().show()   # Cuenta cuantos vuelos retrasados hay (Presencia de dataset desbalanceado)

+-------+------+
|DELAYED| count|
+-------+------+
|      1| 41049|
|      0|140244|
+-------+------+



Esto indica que estamos trabajando con un dataset desbalanceado, por lo que debemos utlizar métodos para balancear el dataset.

In [133]:
# Objetivo de balanceo
target_size = 1125

# Separar las clases
df_majority = df_clean.filter(col("DELAYED") == 0)
df_minority = df_clean.filter(col("DELAYED") == 1)

# Submuestreo de clase mayoritaria (clase 0)
df_majority_sampled = df_majority.sample(False, fraction=target_size / df_majority.count(), seed=42)

# Sobremuestreo de clase minoritaria (clase 1)
ratio = int(target_size / df_minority.count())
df_minority_oversampled = df_minority
for _ in range(ratio - 1):
    df_minority_oversampled = df_minority_oversampled.union(df_minority)

# Agregar una fracción adicional si no es exacto
remaining = target_size - df_minority_oversampled.count()
if remaining > 0:
    df_minority_oversampled = df_minority_oversampled.union(
        df_minority.sample(withReplacement=True, fraction=remaining / df_minority.count(), seed=42)
    )

# Unir los datasets balanceados
df_balanced = df_majority_sampled.union(df_minority_oversampled)

# Verificación
df_balanced.groupBy("DELAYED").count().show()

+-------+-----+
|DELAYED|count|
+-------+-----+
|      0| 1211|
|      1|41049|
+-------+-----+



In [134]:
# Resumen final del dataset balanceado
df_balanced.describe().toPandas()

,summary,CRS_DEP_TIME,CRS_ARR_TIME,CRS_ELAPSED_TIME,DISTANCE,ARR_DELAY,DELAYED,FL_DAY_OF_WEEK,OP_CARRIER_T,ORIGIN_T,DEST_T
0,count,42260,42260,42260,42260,42260,42260,42260,42260,42260,42260
1,mean,1423.4956223379081,1576.102910553715,144.54955040227165,838.6756980596308,51.13279697113109,0.9713440605773781,3.723805016564127,4.0995740653099855,32.02915286322764,33.808944628490295
2,stddev,473.9276427306872,518.0306945564705,77.82770758538584,632.411128240564,70.04188095528303,0.16683954935625955,2.0824270298123135,4.1968499609772,46.7565226995884,46.715027491013444
3,min,1,1,24.0,31.0,-68.0,0,1,0.0,0.0,0.0
4,max,2359,2359,685.0,4983.0,1568.0,1,7,17.0,355.0,355.0


## **Modelo No Supervisado**

In [135]:
# Selecciona columnas relevantes para clustering
clustering_cols = [
    "OP_CARRIER", "ORIGIN", "DEST",
    "CRS_DEP_TIME", "CRS_ARR_TIME", "CRS_ELAPSED_TIME",
    "DISTANCE", "CANCELLED", "DIVERTED",
    "ARR_DELAY"
]

df_clustering = muestra_final.select(clustering_cols)

In [136]:
# Codificar variables categóricas
indexers = [
    StringIndexer(inputCol="OP_CARRIER", outputCol="OP_CARRIER_T"),
    StringIndexer(inputCol="ORIGIN", outputCol="ORIGIN_T"),
    StringIndexer(inputCol="DEST", outputCol="DEST_T")
]

df_indexed = df_clustering
for indexer in indexers:
    df_indexed = indexer.fit(df_indexed).transform(df_indexed)

# Columnas finales para el VectorAssembler (numéricas y las transformadas)
feature_cols = [col for col in df_indexed.columns if col not in clustering_cols] + ["CANCELLED", "DIVERTED", "CRS_DEP_TIME", "CRS_ARR_TIME", "CRS_ELAPSED_TIME", "DISTANCE", "ARR_DELAY"]


# **4. Prepraración del conjunto de entrenamiento y prueba**

## **Modelo Supervisado**

In [143]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=[
        "OP_CARRIER_T","ORIGIN_T","DEST_T",
        "CRS_DEP_TIME","CRS_ARR_TIME","CRS_ELAPSED_TIME","DISTANCE",
        "FL_DAY_OF_WEEK",
        #"ARR_DELAY","DELAYED"
    ],
    outputCol="features"
)

df_vector = assembler.transform(df_balanced)

In [144]:
# Separación de datos de entrenamiento y prueba
spark.conf.set("spark.sql.shuffle.partitions", "200")       # Se define el valor por default del número de ejecutores
df_train, df_val_test = df_vector.randomSplit([0.7, 0.3], seed=42)
df_val, df_test = df_val_test.randomSplit([0.5, 0.5], seed=42)

# Impresion del tamaño de las particiones
print(f"""Total data: {df_vector.count()}
Training data: {df_train.count()}
Validation data: {df_val.count()}
Test data: {df_test.count()}""")

Total data: 42260
Training data: 29554
Validation data: 6363
Test data: 6343


## **Modelo No Supervisado**

In [139]:
# Ensamblar las características en un vector
assembler = VectorAssembler(
    inputCols=feature_cols,
    outputCol="features"
)

df_vector_clustering = assembler.transform(df_indexed)

# Mostrar el schema del dataframe con el vector de características
df_vector_clustering.printSchema()

root
 |-- OP_CARRIER: string (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- CRS_DEP_TIME: integer (nullable = true)
 |-- CRS_ARR_TIME: integer (nullable = true)
 |-- CRS_ELAPSED_TIME: double (nullable = true)
 |-- DISTANCE: double (nullable = true)
 |-- CANCELLED: double (nullable = true)
 |-- DIVERTED: double (nullable = true)
 |-- ARR_DELAY: double (nullable = true)
 |-- OP_CARRIER_T: double (nullable = false)
 |-- ORIGIN_T: double (nullable = false)
 |-- DEST_T: double (nullable = false)
 |-- features: vector (nullable = true)



# **5. Construcción de modelos de aprendizaje supervisado y no supervisado**

### Modelo Supervisado - Regresión Logística

#### Entrenando el modelo

In [145]:
from pyspark.ml.classification import GBTClassifier

# Inicializar el modelo GBTClassifier
gbt = GBTClassifier(featuresCol="features", labelCol="DELAYED", maxIter=10, maxBins=400) # Ajusta maxBins a un valor mayor o igual al máximo de categorías

# Entrenar el modelo
gbt_model = gbt.fit(df_train)

#### Evaluando el modelo

In [146]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

# Realizar predicciones en el conjunto de prueba
predictions = gbt_model.transform(df_test)

# Inicializar el evaluador
evaluator_auc = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol="DELAYED")


# Calcular el área bajo la curva ROC (AUC)
auc = evaluator_auc.evaluate(predictions)
print(f"Área bajo la curva ROC (AUC): {auc}")


# Inicializar el evaluador para otras métricas (MulticlassClassificationEvaluator)
evaluator_multi = MulticlassClassificationEvaluator(labelCol="DELAYED", predictionCol="prediction", metricName="accuracy")

# Calcular Accuracy
accuracy = evaluator_multi.evaluate(predictions)
print(f"Accuracy: {accuracy}")

# Calcular F1-Score
evaluator_multi.setMetricName("f1")
f1_score = evaluator_multi.evaluate(predictions)
print(f"F1-Score: {f1_score}")

# Calcular Precision (para la clase positiva, asumiendo 1 es la clase positiva)
evaluator_multi.setMetricName("weightedPrecision") # Weighted average precision
precision = evaluator_multi.evaluate(predictions)
print(f"Weighted Precision: {precision}")

# Calcular Recall (para la clase positiva)
evaluator_multi.setMetricName("weightedRecall") # Weighted average recall
recall = evaluator_multi.evaluate(predictions)
print(f"Weighted Recall: {recall}")

Área bajo la curva ROC (AUC): 0.5595683315563575
Accuracy: 0.9711492984392244
F1-Score: 0.9577118171443362
Weighted Precision: 0.9446411209968157
Weighted Recall: 0.9711492984392244


#### Conlusiones Modelo Supervisado

De acuerdo a las métricas obtenidas pode

### Modelo No Supervisado - K-Means

#### Entrenando el modelo

In [ ]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

# Define el número de clústeres (puedes experimentar con diferentes valores)
k = 5

# Inicializa el modelo K-Means
kmeans = KMeans(featuresCol="features").setK(k).setSeed(1)

# Entrena el modelo
model = kmeans.fit(df_vector_clustering)

#### Evaluando el modelo

In [ ]:
# Realiza predicciones (asigna cada fila a un clúster)
predictions = model.transform(df_vector_clustering)

# Mostrar algunas predicciones
predictions.select("features", "prediction").show(10)

# Evaluar el modelo usando Silhouette Score
evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(predictions)
print(f"Silhouette with squared Euclidean distance = {silhouette}")

# Muestra los centros de los clústeres
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)